In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('input/train.csv')

In [3]:
test = pd.read_csv('input/test.csv')

### Подготовка train и test

#### Floor

train

In [4]:
#Выделить 1 этаж
train['Floor_First'] = (train['Floor'] == 1).astype(int)

In [5]:
#Если Floor больше HouseFloor - поменять HouseFloor
train.loc[train['Floor'] > train['HouseFloor'], 'HouseFloor'] \
= train.loc[train['Floor'] > train['HouseFloor'], 'Floor'] + 2 

In [6]:
train.loc[train['Square'] < 10, 'Square'] = train.loc[train['Square'] < 10, 'Square'] * 10

test

In [7]:
#Выделить 1 этаж
test['Floor_First'] = (test['Floor'] == 1).astype(int)

In [8]:
#Если Floor больше HouseFloor - поменять HouseFloor
test.loc[test['Floor'] > test['HouseFloor'], 'HouseFloor'] \
= test.loc[test['Floor'] > test['HouseFloor'], 'Floor'] + 2 

#### HouseFloor

train


In [9]:
#Дома выше 50 этажей считать ошибкой
train.loc[train['HouseFloor'] > 50, 'HouseFloor'] = 17

test

In [10]:
#Проблем нет

#### Ecology_1

train

In [11]:
#Заполнить нулевые средним по району
arr_distr = train.loc[train['Ecology_1'] == 0, 'DistrictId'].unique()

In [12]:
mean_Ecology_1_distr = train.loc[(train['DistrictId'].isin(arr_distr)) & (train['Ecology_1'] != 0)]\
.groupby('DistrictId', as_index=False)[['Ecology_1']].mean()

mean_Ecology_1_distr.head()

,DistrictId,Ecology_1
0,2,0.181929
1,7,0.093072
2,17,0.065707
3,26,0.155053
4,31,0.151346


In [13]:
for index, row in mean_Ecology_1_distr.iterrows():
    train.loc[(train['DistrictId'] == row.DistrictId) & (train['Ecology_1'] == 0), 'Ecology_1'] = row.Ecology_1
    

In [14]:
#Если в районе все нулевые, то средним по всему списку
train.loc[train['Ecology_1'] == 0, 'Ecology_1'] = train.loc[train['Ecology_1'] != 0, 'Ecology_1'].mean()

test

In [15]:
for index, row in mean_Ecology_1_distr.iterrows():
    test.loc[(test['DistrictId'] == row.DistrictId) & (test['Ecology_1'] == 0), 'Ecology_1'] = row.Ecology_1

In [16]:
#Если в районе все нулевые, то средним по всему списку
test.loc[test['Ecology_1'] == 0, 'Ecology_1'] = train.loc[train['Ecology_1'] != 0, 'Ecology_1'].mean()

#### Square

train

In [17]:
#Анализ больших площадей
train.loc[train['Square'] > 200, ['Id', 'Square', 'LifeSquare', 'Price']]

,Id,Square,LifeSquare,Price
1981,7917,212.932361,211.231125,302211.260887
1982,5548,275.645284,233.949309,455264.882666
4262,28,604.705972,NaN,187717.242538
4690,2307,409.425181,410.639749,90470.430830
6977,11602,641.065193,638.163193,133529.681562
9910,16568,200.334539,201.627361,528560.506016


In [18]:
train.loc[train['Id'] == 14990, 'Square'] = 163.542020
train.loc[train['Id'] == 28, 'Square'] = 60.705972
train.loc[train['Id'] == 16550, 'Square'] = 74.805921
train.loc[train['Id'] == 2307, 'Square'] = 41.063974
train.loc[train['Id'] == 11602, 'Square'] = 138.163193
train.loc[train['Id'] == 15886, 'Square'] = 161.463614

In [19]:
#Анализ маленьких площадей
train.loc[train['Square'] < 10, ['Id', 'Rooms', 'Square', 'LifeSquare', 'Price']]

,Id,Rooms,Square,LifeSquare,Price


In [20]:
train.loc[train['Square'] < 10, 'Square'] = train.loc[train['Square'] < 10, 'Square'] * 10

test

In [21]:
#Анализ больших площадей
test.loc[test['Square'] > 200, ['Id', 'Rooms', 'Square', 'LifeSquare']]

,Id,Rooms,Square,LifeSquare
3217,4058,6.0,223.453689,104.113552
3909,1397,3.0,221.138768,118.055342


In [22]:
#Анализ маленьких площадей
test.loc[test['Square'] < 10, ['Id', 'Rooms', 'Square', 'LifeSquare']]

,Id,Rooms,Square,LifeSquare
66,9011,1.0,1.378543,1.353573
608,16401,1.0,2.645046,4.338755
837,2138,1.0,5.647458,1.501582
1165,10120,1.0,5.100672,3.861780
4490,1165,1.0,2.372101,1.899119
4540,7855,1.0,4.967143,2.968086
4713,170,2.0,2.900586,61.468563
4902,1420,1.0,1.975769,2.900371


In [23]:
test.loc[test['Square'] < 10, 'Square'] = test.loc[test['Square'] < 10, 'Square'] * 10

#### LifeSquare

train

In [24]:
#Найти среднее отношение
d1 = train.loc[(train['LifeSquare'].isnull() == False) \
                 & (train['Square'] > train['LifeSquare']), ['Square', 'LifeSquare']]

coef = (d1['Square'] / d1['LifeSquare']).median()

train.loc[(train['LifeSquare'].isnull()), 'LifeSquare'] \
= train.loc[train['LifeSquare'].isnull(), 'Square'] / coef

In [25]:
#Изменить странные отношения
train.loc[train['Square'] / train['LifeSquare'] > 2.2, 'LifeSquare']\
= train.loc[train['Square'] / train['LifeSquare'] > 2.2, 'Square'] / coef

train.loc[train['Square'] / train['LifeSquare'] <= 1, 'LifeSquare']\
= train.loc[train['Square'] / train['LifeSquare'] <= 1, 'Square'] / coef


test

In [26]:
test.loc[(test['LifeSquare'].isnull()), 'LifeSquare'] \
= test.loc[test['LifeSquare'].isnull(), 'Square'] / coef

In [27]:
test.loc[test['Square'] / test['LifeSquare'] > 2.2, 'LifeSquare']\
= test.loc[test['Square'] / test['LifeSquare'] > 2.2, 'Square'] / coef

test.loc[test['Square'] / test['LifeSquare'] <= 1, 'LifeSquare']\
= test.loc[test['Square'] / test['LifeSquare'] <= 1, 'Square'] / coef

#### Rooms

train

In [28]:
#Средняя площадь по количеству комнат
train.groupby('Rooms', as_index=False)[['Square']].mean()

,Rooms,Square
0,0.0,84.294762
1,1.0,41.290226
2,2.0,56.560807
3,3.0,76.974573
4,4.0,98.377544
5,5.0,122.614941
6,6.0,59.414334
7,10.0,59.964120
8,19.0,42.006046


In [29]:
#0
train.loc[train['Rooms'] == 0, ['Id', 'Square']]

,Id,Square
1397,12638,138.427694
1981,7917,212.932361
2269,7317,41.790881
3911,770,49.483501
4366,456,81.491446
4853,3224,23.772476
6149,3159,38.697117
8834,9443,87.762616


In [30]:
#Заполнение
train.loc[train['Id'] == 12638, 'Rooms'] = 5
train.loc[train['Id'] == 7917, 'Rooms'] = 5
train.loc[train['Id'] == 7317, 'Rooms'] = 1
train.loc[train['Id'] == 770, 'Rooms'] = 2
train.loc[train['Id'] == 456, 'Rooms'] = 4
train.loc[train['Id'] == 3224, 'Rooms'] = 1
train.loc[train['Id'] == 3159, 'Rooms'] = 1
train.loc[train['Id'] == 9443, 'Rooms'] = 4

In [31]:
#6
train.loc[train['Rooms'] == 6, ['Id', 'Square']]

,Id,Square
2170,14003,59.414334


In [32]:
train.loc[train['Id'] == 14003, 'Rooms'] = 2

In [33]:
#10
train.loc[train['Rooms'] == 10, ['Id', 'Square']]

,Id,Square
377,5927,59.056975
8849,14865,60.871266


In [34]:
#Заполнение
train.loc[train['Id'] == 5927, 'Rooms'] = 2
train.loc[train['Id'] == 14865, 'Rooms'] = 2

In [35]:
#19
train.loc[train['Rooms'] == 19, ['Id', 'Square']]

,Id,Square
1454,8491,42.006046


In [36]:
train.loc[train['Id'] == 8491, 'Rooms'] = 1

test

In [37]:
test['Rooms'].value_counts()

2.0     2030
1.0     1769
3.0     1099
4.0       90
5.0        7
0.0        2
6.0        2
17.0       1
Name: Rooms, dtype: int64

In [38]:
#0
test.loc[test['Rooms'] == 0, ['Id', 'Square']]

,Id,Square
2406,3343,116.824201
2524,10729,76.345154


In [39]:
#Заполнение
test.loc[test['Id'] == 3343, 'Rooms'] = 5
test.loc[test['Id'] == 10729, 'Rooms'] = 3

In [40]:
#6
test.loc[test['Rooms'] == 6, ['Id', 'Square']]

,Id,Square
2071,10793,110.750226
3217,4058,223.453689


In [41]:
#Заполнение
test.loc[test['Id'] == 10793, 'Rooms'] = 5
test.loc[test['Id'] == 4058, 'Rooms'] = 5

In [42]:
#17
test.loc[test['Rooms'] == 17, ['Id', 'Square']]

,Id,Square
3398,1435,52.866107


In [43]:
#Заполнение
test.loc[test['Id'] == 1435, 'Rooms'] = 2

#### HouseYear

train

In [44]:
train.loc[train['HouseYear'] == 4968, 'HouseYear'] = 1968
train = train.drop(train.loc[train['HouseYear'] == 20052011].index, axis = 0)

In [45]:
#Группировка годов
years = np.array([1900,2005, 2010, 2015, 2030])

for i in np.arange(1, years.shape[0]):
    
    train.loc[(years[i-1] <= train['HouseYear']) & (train['HouseYear'] < years[i]), 'HouseYear']\
    = years[i-1]
    
train['HouseYear'] = train['HouseYear'].astype(str)
train = pd.get_dummies(train)

test

In [46]:
#Группировка годов

for i in np.arange(1, years.shape[0]):
    
    test.loc[(years[i-1] <= test['HouseYear']) & (test['HouseYear'] < years[i]), 'HouseYear']\
    = years[i-1]
    
test['HouseYear'] = test['HouseYear'].astype(str)
test = pd.get_dummies(test)

#### DistrictId

train

In [47]:
#DistrictId заменить на среднюю цену за кв метр по району
train['mean_price_meter_distr'] = train['Price'] / train['Square']

In [48]:
#Перекос
#train.drop(train.loc[train['mean_price_meter_distr'] > 9000].index, axis=0, inplace=True)

In [49]:
mean_distr_Price = train.groupby('DistrictId', as_index=False)[['mean_price_meter_distr']].mean()

In [50]:
train = pd.merge(train, mean_distr_Price, on='DistrictId', how='inner')
train.drop('DistrictId', axis=1, inplace = True)
train.drop('mean_price_meter_distr_x', axis=1, inplace = True)
train.rename(columns={'mean_price_meter_distr_y':'mean_price_meter_distr'}, inplace=True)
#train.head()

test

In [51]:
test = pd.merge(test, mean_distr_Price, on='DistrictId', how='left')

In [65]:
#Районы, которых нет - средним по Колву комнат
Rooms_null = test.loc[test['mean_price_meter_distr'].isnull(), 'Rooms'].unique()
Rooms_null

array([], dtype=float64)

In [53]:
tmp_df = train.loc[train['Rooms'].isin(Rooms_null)].groupby('Rooms', as_index=False)[['Price']].mean()
tmp_df

,Rooms,Price
0,2.0,215275.678371
1,3.0,289337.962452
2,4.0,381051.393819


In [54]:
for index, row in tmp_df.iterrows():
    test.loc[(test['mean_price_meter_distr'].isnull()) & (test['Rooms'] == row.Rooms), 'mean_price_meter_distr'] \
    = row.Price

#### Shops_1

train

In [55]:
train.loc[train['Shops_1'] > 0, 'Shops_1'] = 1

test

In [56]:
test.loc[test['Shops_1'] > 0, 'Shops_1'] = 1

### Построение модели

In [58]:
feats = ['Rooms', 'Square', 'LifeSquare', 'Floor', 'Floor_First',\
         'HouseFloor','Ecology_1', 'Shops_1', 'HouseYear_2005', 'HouseYear_2010',\
         'mean_price_meter_distr']

In [68]:
X_test = test[feats]

In [66]:
X_train = train[feats]
y_train = train['Price']

In [67]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9998
Data columns (total 11 columns):
Rooms                     9999 non-null float64
Square                    9999 non-null float64
LifeSquare                9999 non-null float64
Floor                     9999 non-null int64
Floor_First               9999 non-null int32
HouseFloor                9999 non-null float64
Ecology_1                 9999 non-null float64
Shops_1                   9999 non-null int64
HouseYear_2005            9999 non-null uint8
HouseYear_2010            9999 non-null uint8
mean_price_meter_distr    9999 non-null float64
dtypes: float64(6), int32(1), int64(2), uint8(2)
memory usage: 761.6 KB


In [60]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [69]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [70]:
pred.shape

(5000,)

In [71]:
result = pd.DataFrame(data = {'Id':test['Id'], 'Price':pred})
result = result.set_index('Id') #чтобы было только два поля
result.head()

,Price
Id,
725,145146.091025
15856,208863.590731
5480,300010.371235
15664,312548.255561
14275,123347.803149


In [64]:
result.to_csv('output/PDolgov_predictions.csv')